## 오픈 언어모델 Fine-Tuning 이론
___

출처 : https://www.catalyzex.com

<img src="https://d3i71xaburhd42.cloudfront.net/38258a93151d57a073fe5cfccefd443863942478/2-Figure1-1.png" width="800">



### 1. Fine-Tuning 의 개념

사전 학습된 모델을 특정 도메인이나 서비스 상황에 맞게 최적화하는 방법

새로운 데이터셋에 대한 특성을 사전 학습된 모델을 통해 추출하고, 모델의 출력 층을 새로운 작업에 맞게 조정한 후, 전체 모델을 새로운 데이터셋에 맞게 재학습하는 과정

**절차는 사전 학습된 모델 로드 → 새로운 데이터셋의 특성 추출 → 새로운 분류기 추가 → 미세조정 를 거침**

### 2. Fine-Tuning 이 필요한 상황

1. 커스터마이즈 : 특정 도메인이나 서비스 상황에 최적화 해야 할때
2. 부족한 라벨 데이터 보완 :  사전 학습된 모델의 유용한 하위계층 데이터를 활용

### 3. Fune-Tuning의 어려운점

1. **도메인 차이(Domain Shift)**:
    - **문제**: 사전 학습된 모델이 학습한 데이터와 새로운 데이터 간에 차이가 클 경우, 모델이 기대한 대로 동작하지 않을 수 있음.
    - **설명**: 예를 들어, 자연 이미지로 학습된 모델을 의료 이미지에 적용하려 할 때, 데이터 특성이 크게 다르면 성능이 저하됨.
2. **과적합(Overfitting)**:
    - **문제**: 새로운 데이터셋이 작을 경우, 미세 조정 과정에서 모델이 과적합될 위험이 높아짐
    - **설명**: 모델이 새로운 데이터셋에 너무 맞추어져, 일반화 능력이 떨어짐.
3. **학습률 조절**:
    - **문제**: 학습률을 잘못 설정하면, 기존의 학습된 가중치가 크게 변하거나, 학습이 제대로 되지 않을 수 있음.
    - **설명**: 너무 높은 학습률은 모델을 불안정하게 만들고, 너무 낮은 학습률은 학습 속도를 느리게 만듦
4. **컴퓨팅 자원과 시간**:
    - **문제**: 큰 모델을 미세 조정하는 데는 많은 계산 자원과 시간이 필요함.
    - **설명**: 특히 대규모 모델의 경우, 하드웨어 요구사항이 매우 높아 개인/중소 규모에서 수행하기 어려워짐.

### 4. PEFT(Parameter Efiicient Fine Tuning) 의 장점

1. **효율적인 학습**
    - PEFT는 모델의 일부 파라미터만 조정하기 때문에, 전체 모델을 학습하는 것보다 훨씬 효율적
    - 메모리 사용과 계산 자원을 절약할 수 있어, 더 적은 자원으로도 높은 성능을 달성 가능
2. **저비용 고효율**
    - 적은 학습 데이터와 자원으로도 모델을 효과적으로 미세 조정 가능
    - 대규모 언어 모델과 같이 큰 모델을 사용할 때 유리
3. **빠른 학습**
    - 특정 파라미터만 조정하므로, 학습 속도가 빠름.
    - 실시간 응용에도 적합
4. **적응력**
    - 모델의 특정 부분을 조정하는 다양한 방법을 제공되고 있어, 다양한 도메인과 작업에 Adapting 할수 있음.
    - Adapters, Low-Rank Adaptation (LoRA), Prefix-Tuning 등의 기법을 통해, 필요한 부분만 조정 가능.
5. **일관된 성능과 안정성**
    - PEFT는 모델의 주요 특성을 유지하면서도 특정 작업에 맞는 조정을 할 수 있어, 안정적인 성능을 제공
    - 과적합의 위험을 줄이고, 일반화 능력을 유지하게 됨.

### 5. LLM 분야의 대표적 PEFT 방법론

- 참고 문서
    - https://medium.com/@abonia/llm-series-parameter-efficient-fine-tuning-e9839fae44ac
    - https://lightning.ai/pages/community/article/lora-llm/
    - https://huggingface.co/docs/transformers/peft
    - https://github.com/Lightning-AI/lit-llama/ (fine-tune 구현 코드)
1. **LoRA(Low Rank Adaptation)**
    
    [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
    
    !https://lightningaidev.wpengine.com/wp-content/uploads/2023/04/lora-5-1024x511.png
    
    1. LoRA는 Low-Rank Factorizaton 방법을 활용해 LLM의 Linear Layer에 대한 업데이트를 근사화하는 기술.
    2. 기본 원리는 모델의 고차원 파라미터를 저차원 공간으로 투영한뒤, 저차원 공간에서의 추가 파라미터만 학습하고 이를 원래 모델에 병합하여 출력을 조절.
    3. 고차원 파라미터 대신 저차원 파라미터를 학습하므로, 필요한 메모리와 계산 자원이 크게 줄어들어,  모델의 최종 성능에 거의 영향을 주지 않으면서 훈련 속도를 높임
    4. 알고리즘 설명 (Transformer 에시)
        - 원래 가중치 행렬 W: 크기가 d x k인 행렬
        - 저차원 행렬 A:  크기가 d x r 인 행렬로, 저차원 공간으로 투영.
        - 저차원 행렬 B: 크기가 r x k 인 행렬로, 다시 고차원 공간으로 변환.
        - 학습: 학습 과정에서 A와 B 행렬을 학습. 이때 r은 d나 k보다 훨씬 작기 때문에, 필요한 파라미터 수가 크게 줄어듬.
        - 출력 계산: 학습된 저차원 행렬을 사용하여, 원래 가중치 행렬의 출력을 조정.
        
        $$
         W +\alpha \cdot (A \cdot B)
        $$
        
    5. QLoRA : 기존 RoLA에서 양자화 기법으로 응용한 형태로 계산 효율성을 한단계 더 극대화. (저차원 행렬의 값을 정밀도가 낮은 데이터 타입(예: 8-bit 정수)으로 변환하여 저장하고 연산)
    6. 예시 코드
        
        ```python
        import torch
        import torch.nn as nn
        from transformers import AutoModelForSequenceClassification
        
        class LoRALayer(nn.Module):
            def __init__(self, original_layer, rank):
                super().__init__()
                self.original_layer = original_layer
                self.rank = rank
                self.lora_A = nn.Parameter(torch.randn((original_layer.weight.size(0), rank)))
                self.lora_B = nn.Parameter(torch.randn((rank, original_layer.weight.size(1))))
        
            def forward(self, x):
                return self.original_layer(x) + (x @ self.lora_B.t() @ self.lora_A.t())
        
        model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
        for name, module in model.named_modules():
            if isinstance(module, nn.Linear):
                setattr(model, name, LoRALayer(module, rank=8))
        
        # 학습 과정은 일반적인 방식과 동일합니다.
        ```
        
2. **Prompt Tuning**
    
    [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/abs/2104.08691)
    
    1. 모델의 가중치를 변경하지 않고, 입력 데이터에 대한 임베딩을 수정하여 모델이 원하는 작업을 수행
    2. 구현 절차 설명
        - **기존 모델 준비**
            - 미리 학습된 대형 언어 모델(예: GPT, BERT)을 사용합니다.
        - **소프트 프롬프트 생성 (자연어가 아닌 특정 목적에  최적화된 훈련가능한 임베딩 벡터)**
            - 모델의 입력 임베딩 차원과 동일한 차원의 임의의 초기값을 가진 소프트 프롬프트 벡터를 생성.
        - **입력 데이터와 소프트 프롬프트 결합**
            - 입력 데이터 앞에 소프트 프롬프트 벡터를 추가하여 모델에 입력.\
        - **학습 과정**
            - 모델을 학습하면서 소프트 프롬프트 벡터를 최적화. 이때 모델의 나머지 부분은 고정된 상태로 유지.
            - 일반적인 뉴럴 네트워크 최적화 방법(SGD)을 사용해 역전파 학습
    3. 예시 코드
        
        ```python
        import torch
        from torch import nn
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        
        class PromptTuningModel(nn.Module):
            def __init__(self, model_name, prompt_length):
                super(PromptTuningModel, self).__init__()
                self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
                self.prompt_embeddings = nn.Parameter(torch.randn(prompt_length, self.model.config.hidden_size))
        
            def forward(self, input_ids, attention_mask):
                batch_size = input_ids.size(0)
                prompt_embeds = self.prompt_embeddings.unsqueeze(0).expand(batch_size, -1, -1)
                
                inputs_embeds = self.model.get_input_embeddings()(input_ids)
                inputs_embeds = torch.cat((prompt_embeds, inputs_embeds), dim=1)
                
                extended_attention_mask = torch.cat((torch.ones(batch_size, self.prompt_embeddings.size(0), device=input_ids.device), attention_mask), dim=1)
                
                outputs = self.model(inputs_embeds=inputs_embeds, attention_mask=extended_attention_mask)
                return outputs
        
        # 모델과 토크나이저 준비
        model_name = "bert-base-uncased"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = PromptTuningModel(model_name, prompt_length=5)
        
        # 입력 데이터 준비
        inputs = tokenizer("This is a test sentence.", return_tensors='pt')
        outputs = model(**inputs)
        
        # 손실 계산 및 역전파
        labels = torch.tensor([1]).unsqueeze(0)  # 예시 레이블
        loss = nn.CrossEntropyLoss()(outputs.logits, labels)
        loss.backward()
        
        # 최적화 단계
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        optimizer.step()
        ```
        
3. **Prefix-Tuning**
    
    [Prefix-Tuning: Optimizing Continuous Prompts for Generation](https://arxiv.org/abs/2101.00190)
    
    1. 기존의 미세 조정 방법들과는 달리, Prefix-Tuning은 모델의 모든 가중치를 학습하는 대신 입력 데이터에 추가적인 벡터(프리픽스)를 붙여서 학습하는 방법
    2. 사전 학습된 언어 모델의 가중치는 고정(frozen)된 상태로 유지시킴. 대신, 프리픽스 벡터만을 추가로 학습시킴. 이를 통해 전체 모델의 가중치를 재학습하는 부담을 줄임
    3. 예를 들어, 입력 문장이 “This movie is great”라면, 프리픽스 벡터가 추가된 입력은 “[프리픽스 벡터] This movie is great”가 되는 것임.
    4. 예시 코드
        
        ```python
        import torch
        import torch.nn as nn
        from transformers import AutoModelForSequenceClassification, AutoTokenizer
        
        class PrefixTuningModel(nn.Module):
            def __init__(self, model_name, prefix_length):
                super().__init__()
                self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
                self.prefix_embeddings = nn.Parameter(torch.randn((self.model.config.num_hidden_layers, prefix_length, self.model.config.hidden_size)))
        
            def forward(self, input_ids, attention_mask):
                prefix_embeds = self.prefix_embeddings.unsqueeze(1).expand(-1, input_ids.size(0), -1, -1)
                outputs = self.model(input_ids, attention_mask, output_hidden_states=True)
                hidden_states = outputs.hidden_states
                for i, hidden_state in enumerate(hidden_states):
                    hidden_states[i] = torch.cat((prefix_embeds[i], hidden_state), dim=1)
                return self.model(inputs_embeds=hidden_states[-1], attention_mask=attention_mask)
        
        tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        model = PrefixTuningModel('bert-base-uncased', prefix_length=5)
        
        # 입력 데이터 준비
        inputs = tokenizer("This is a test sentence.", return_tensors='pt')
        outputs = model(**inputs)
        ```
        

[Prefix-Tuning: Optimizing Continuous Prompts for Generation](https://arxiv.org/abs/2101.00190)

## 오픈 언어모델 Fine-Tuning 실습
___

* ref
  * https://huggingface.co/docs/transformers/training

  * https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M

* https://lightning.ai/ 에서 진행할 예정입니다.
* 무료 회원가입후, A10G 이상에서 진행합니다.
* 구글 Colab 도 가능하지만, 구글드라이브에 데이터셋을 준비후 마운트해야 가능합니다.

In [ ]:
#!pip install transformers datasets bitsandbytes peft trl accelerate --upgrade -qqq

In [5]:
import torch
import os
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments, Trainer
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import wandb
from glob import glob
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

* 훈련 데이터 준비

In [9]:
files = glob('dataset/consult/*.csv')

df = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

dataset = Dataset.from_pandas(df)

dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 577
})

In [10]:
dataset.to_pandas().head()

,question,answer
0,AB테스트란 무엇인가요?,AB테스트는 두 가지 이상의 변형을 비교하여 어떤 변형이 더 효과적인지를 판단하는 ...
1,AB테스트를 진행할 때 고려해야 할 요소는 무엇인가요?,"AB테스트를 진행할 때는 목표 설정, 샘플 크기 결정, 랜덤화, 통계적 가설 검정,..."
2,AB테스트에서 유의수준과 검정력의 중요성은 무엇인가요?,"유의수준은 가설이 참일 때 오류를 범할 확률을 나타내며, 검정력은 대립가설이 참일 ..."
3,AB테스트 결과 해석 시 어떤 점을 주의해야 하나요?,AB테스트 결과를 해석할 때는 통계적 유의성 뿐만 아니라 실제 비즈니스 영향성도 함...
4,AB테스트를 통해 얻은 결과를 어떻게 활용할 수 있나요?,AB테스트를 통해 얻은 결과를 바탕으로 제품 또는 서비스의 개선을 시도할 수 있으며...


* 프롬프팅 함수 정의

In [ ]:
# LLAMA 계열 프롬프팅 함수 (모델 계열에 따라 달라짐)
# 이 함수는 주어진 예제 데이터에서 사용자와 어시스턴트 간의 대화 형식으로 프롬프트를 생성
# 데이터셋 필드에 따라 필드명은 다르게 해야함. 여기서는 question, answer
def generate_prompts(example):
    prompt_list = []
    for i in range(len(example['question'])):
        prompt_list.append(
f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>다음 글을 요약해주세요:
{example['question'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{example['answer'][i]}<|eot_id|>"""
        )
    return prompt_list

* 양자화 설정

In [ ]:
# Bitsandbytes
# CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼.
# 4비트 정밀도로 저장된 모델을 실행 가능하도록 함. 
# 가중치를 4비트로 저장하지만 계산은 여전히 ​​16비트 또는 32비트로 이루어짐
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

* RoLA 설정

In [ ]:
# LoRA 설정 : 양자화된 모델에서 Adaptor를 붙여서 학습할 파라미터만 따로 구성함
lora_config = LoraConfig(
    r=8,  # LoRA의 랭크를 설정합니다. 이는 모델의 파라미터 수를 줄이기 위해 사용
    lora_alpha = 8,  # LoRA의 스케일링 파라미터로, 학습 속도와 관련
    lora_dropout = 0.05, # 드롭아웃 확률을 설정하여 과적합을 방지
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # LoRA가 적용될 모델의 모듈 리스트를 지정
    bias="none",  # "none"은 편향을 사용하지 않음을 의미
    task_type="CAUSAL_LM", # LoRA가 적용될 작업 유형을 지정
)

* 토크나이저 준비

In [12]:
# base_model에서 사전 학습된 토크나이저를 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_name,  add_special_tokens=True)
# 패딩 토큰을 eos 토큰으로 설정
tokenizer.pad_token = tokenizer.eos_token
# 패딩 방향을 오른쪽으로 설정
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


* 베이스 모델 준비 및 PEFT 설정

In [ ]:
# 사전 학습된 모델 로드 (Llama 계열 모델은 AutoModelForCausalLM를 사용해야함)
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", quantization_config=bnb_config)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=128,
    args=TrainingArguments(
        output_dir="outputs", # 출력 디렉토리
        num_train_epochs = 1, # 학습 에포크 수
        # max_steps=300,   # 최대 스텝 수
        per_device_train_batch_size=4, # GPU당 4개 배치
        gradient_accumulation_steps=4, # gradient 반영을 4개 step 마다
        optim="paged_adamw_8bit", # 옵티마이저
        warmup_steps=500, # 웜업 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=True, # FP16 사용 여부
        logging_steps=100, # 로깅 스텝 수
        push_to_hub=False, # HuggingFace Hub에 푸시 여부
        report_to='none',  # 리포트 대상
    ),
    peft_config=lora_config,   # LoRA 설정값
    formatting_func=generate_prompts,   # 프롬프트 템플릿 함수
)

In [18]:
# 모델 학습 시작
trainer.train()

Step,Training Loss


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=36, training_loss=3.1319512261284723, metrics={'train_runtime': 108.4896, 'train_samples_per_second': 5.318, 'train_steps_per_second': 0.332, 'total_flos': 2541164004605952.0, 'train_loss': 3.1319512261284723, 'epoch': 0.993103448275862})

* 최종 튜닝모델 생성

In [19]:
# LoRA 어댑터 모델 저장 경로
lora_model_name = "Data-Science-Interview-QNA"

# 훈련된 모델을 어댑터 모델로 저장
trainer.model.save_pretrained(lora_model_name)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map='auto')

# 어댑터 모델 로드
base_model = PeftModel.from_pretrained(base_model, lora_model_name, device_map='auto')

# 모델 병합 및 언로드
final_model = base_model.merge_and_unload()

# 최종 모델 저장 (앞에 DanteKwak 부분은 여러분의 네임스페이스에 맞게 바꾸셔야 합니다.)
final_model_name = f'DanteKwak/Llama-3-8B-{lora_model_name}'
final_model.save_pretrained(final_model_name)

* 허깅 페이스에 내 모델 업로드하기
  * https://huggingface.co/new 에서 먼저 모델공간을 만드세요.

In [ ]:
from huggingface_hub import HfApi

# Hugging Face에 업로드를 위한 API 객체 생성
api = HfApi()

# Hugging Face에 모델 업로드
api.upload_folder(
    folder_path=final_model_name,
    repo_id=final_model_name,
    repo_type='model'
)

* 생성된 모델 테스트
  * 양자화 된 상태가 아니기 때문에 서빙하기에는 매우 느립니다.
  * AWQ나 vLLM 혹은 둘다 이용해서 양자화된 상태로 서빙하면 속도가 빨라집니다.

In [ ]:
question = "ABtest에서 가장 중요한것은 무엇인가요?"

pipe = pipeline("text-generation", 
                model=final_model, 
                tokenizer=tokenizer, 
                max_new_tokens=512)

messages = [
    {"role": "user", "content": f"{question}",}
]

prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0,
    top_k=50,
    top_p=0.95,
    # add_special_tokens=True,
    eos_token_id = [ # eos_token_id를 지정하지 않으면 생성 토큰 반복
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
)

print(outputs[0]['generated_text'][len(prompt):])

🤔

In A/B testing (also known as split testing), the most important thing is to ensure that the test is designed and executed correctly to produce reliable and actionable results. Here are some key factors to focus on:

1. **Clear goals and hypotheses**: Define what you want to achieve with the test and what you expect to see. This will help you determine the success metrics and ensure that the test is focused on a specific goal.
2. **Well-defined test variables**: Identify the specific elements you want to test (e.g., button color, headline, CTAs) and ensure that they are well-defined and measurable.
3. **Control group and treatment group**: Ensure that both groups are equally representative of your target audience and that the test is randomized to minimize bias.
4. **Sufficient sample size**: Make sure the test has enough participants to produce statistically significant results. A general rule of thumb is to aim for at least 1,000 participants per group.
5. **Duration of the test**

* Llama.cpp 양자화 ( GGUF 파일 만들기 )
  
  * ref
    * https://github.com/ggerganov/llama.cpp
  
    * https://github.com/ggerganov/llama.cpp/issues/6819
    * https://huggingface.co/docs/hub/gguf

  * 절차
  
    <code>

    git clone https://github.com/ggerganov/llama.cpp

    cd llama.cpp

    pip install -r requirements.txt

    make LLAMA_CUBLAS=1

    python ./convert.py ../DanteKwak/Llama-3-8B-Data-Science-Interview-QNA --outfile ../Llama-3-8B-Data-Science-Interview-QNA.gguf --vocab-type bpe

    ./quantize ./Llama-3-8B-Data-Science-Interview-QNA.gguf ./Llama-3-8B-Data-Science-Interview-QNA.Q4_K_M.gguf q4_k_m

    </code>


In [ ]:
# Hugging Face에 업로드를 위한 API 객체 생성
api = HfApi()

# Hugging Face에 모델 업로드
api.upload_file(
    path_or_fileobj="Llama-3-8B-Data-Science-Interview-QNA.Q4_K_M.gguf",
    path_in_repo="Llama-3-8B-Data-Science-Interview-QNA.Q4_K_M.gguf",
    repo_id="DanteKwak/Llama-3-8B-Data-Science-Interview-QNA-GGUF", #모델공간은 미리 만들어야 합니다.
    repo_type="model",
)

* Ollama에 배포하기 위해 Modelfile 생성
  
  * gguf 와 동일한 파일위치에 Modelfile 파일을 만들어 아래 내용을 기입
  
    ```
    FROM Llama-3-8B-Data-Science-Interview-QNA.Q4_K_M.gguf

    TEMPLATE """{{- if .System }}
    <s>{{ .System }}</s>
    {{- end }}
    <s>Human:
    {{ .Prompt }}</s>
    <s>Assistant:
    """

    SYSTEM """상대방의 요청에 최대한 자세하고 전문가 답게 답해주세요. 모든 응답은 한글로 해주세요."""

    PARAMETER temperature 0
    PARAMETER num_predict 3000
    PARAMETER num_ctx 4096
    PARAMETER stop <s>
    PARAMETER stop </s>
    ```

  * Ollama 에 모델추가 및 실행
  
    ```
    ollama create llama3-ds-qna -f Modelfile

    ollama run llama3-ds-qna
    ```

* Ollama 모델을 Langchain에 통합하여 질의

In [ ]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# Ollama LLM 객체 생성
llm_ds_qna = Ollama(model="llama3-ds-qna")

# 질의 예시
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)
chain = prompt | llm_ds_qna

chain.invoke({"question": "ABtest에서 가장 중요한것은 무엇인가요?"})

* AWQ(AI Weight Quantization)를 이용한 양자화
  
  * 모델의 가중치를 양자화하여 모델의 크기와 연산 속도를 최적화하는 기술
  
  * 신경망의 가중치를 정밀도가 낮은 형식으로 변환하여 메모리 사용을 줄이고 계산 속도를 높이는 방법
  * 양자화 기법:
	  * 균등 양자화(Uniform Quantization): 가중치를 일정 간격으로 나누어 양자화
	  * 비균등 양자화(Non-Uniform Quantization): 데이터 분포에 따라 가중치 간격을 다르게 설정하여 양자화

In [ ]:
from awq import AutoAWQForCausalLM
quant_path = f'DanteKwak/{final_model_name}-AWQ'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# 모델 로드
final_model_for_awq = AutoAWQForCausalLM.from_pretrained(final_model_name, **{"low_cpu_mem_usage": True})
tokenizer = AutoTokenizer.from_pretrained(final_model_name, trust_remote_code=True)

# 양자화
final_model_for_awq.quantize(tokenizer, quant_config=quant_config)

# 양자화 된 모델 저장
final_model_for_awq.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

# 모델 업로드
api.upload_folder(
    folder_path=quant_path,
    repo_id=quant_path,
    repo_type='model'
)